<a href="https://colab.research.google.com/github/dhruvchopra2003/Agent_Staffing/blob/main/Agent_Staffing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.9 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123589 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

# Current Solution

In [3]:
from pyomo.environ import *

# Data
num_shifts = 3
time_slots_per_shift = 8
time_slots = range(num_shifts * time_slots_per_shift)
call_volume_forecast = [50, 40, 45, 30, 25, 20, 15, 10,
                        20, 30, 40, 50, 60, 70, 80, 90,
                        0, 10, 1, 1, 40, 50, 60, 70]
cost_per_agent = 100  # $ per agent per shift
max_agents_per_shift = 100  # Maximum number of agents allowed per shift
total_budget = 1500
agent_ability = 10  # Number of calls an agent can handle per time slot
call_revenue = 30  # Revenue generated per call answered

# Weights for the objective function
weight_profit = 0.5
weight_calls = 0.5

# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(range(num_shifts), within=NonNegativeIntegers)  # Number of agents per shift
model.calls_handled = Var(time_slots, within=NonNegativeReals)  # Number of calls handled per time slot

# Objective function: Maximize weighted sum of profit and call handling
def objective_rule(model):
    total_revenue = sum(model.calls_handled[i] * call_revenue for i in time_slots)
    total_cost = sum(model.x[j] * cost_per_agent for j in range(num_shifts))
    profit = total_revenue - total_cost
    total_calls_handled = sum(model.calls_handled[i] for i in time_slots)
    return weight_profit * profit + weight_calls * total_calls_handled

model.obj = Objective(rule=objective_rule, sense=maximize)

# Service level constraint
def service_level_constraint(model, i):
    shift = i // time_slots_per_shift
    return model.calls_handled[i] <= model.x[shift] * agent_ability

# Ensure all calls are handled if possible
def calls_handled_constraint(model, i):
    return model.calls_handled[i] <= call_volume_forecast[i]

# Max agents per shift constraint
def max_agents_constraint(model, j):
    return model.x[j] <= max_agents_per_shift

# Budget constraint
def budget_constraint(model):
    return sum(model.x[j] * cost_per_agent for j in range(num_shifts)) <= total_budget

# Apply constraints
model.service_level = Constraint(time_slots, rule=service_level_constraint)
model.calls_handled_constraint = Constraint(time_slots, rule=calls_handled_constraint)
model.max_agents = Constraint(range(num_shifts), rule=max_agents_constraint)
model.budget = Constraint(rule=budget_constraint)

# Solve the model using GLPK
solver = SolverFactory('glpk')
result = solver.solve(model, tee=False)

# Check solver status and termination condition
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    print("\nOptimal solution found.")
    # Print results
    print("\nOptimal Number of Agents per Shift:")
    for j in range(num_shifts):
        print(f"Shift {j+1}: {model.x[j].value} agents")

    total_cost = sum(cost_per_agent * model.x[j].value for j in range(num_shifts))
    total_revenue = sum(model.calls_handled[i].value * call_revenue for i in time_slots)
    profit = total_revenue - total_cost
    total_calls_handled = sum(model.calls_handled[i].value for i in time_slots)
    print(f"\nTotal Staffing Cost: ${total_cost:.2f}")
    print(f"Total Revenue: ${total_revenue:.2f}")
    print(f"Profit: ${profit:.2f}")
    print(f"Total Calls Handled: {total_calls_handled:.2f}")

    # Check constraints
    service_level_satisfied = all(model.calls_handled[i].value <= model.x[i // time_slots_per_shift].value * agent_ability for i in time_slots)
    budget_satisfied = total_cost <= total_budget
    max_agents_satisfied = all(model.x[j].value <= max_agents_per_shift for j in range(num_shifts))

    print("\nConstraint Satisfaction:")
    print(f"Service Level Constraints Satisfied: {service_level_satisfied}")
    print(f"Budget Constraint Satisfied: {budget_satisfied}")
    print(f"Max Agents per Shift Constraints Satisfied: {max_agents_satisfied}")

    print("\nSolver Status and Termination Condition:")
    print(f"Solver Status: {result.solver.status}")
    print(f"Termination Condition: {result.solver.termination_condition}")
elif result.solver.termination_condition == TerminationCondition.infeasible:
    print("\nNo feasible solution found.")
else:
    print("\nSolver encountered an issue. Status: ", result.solver.status)



Optimal solution found.

Optimal Number of Agents per Shift:
Shift 1: 4.0 agents
Shift 2: 7.0 agents
Shift 3: 4.0 agents

Total Staffing Cost: $1500.00
Total Revenue: $24060.00
Profit: $22560.00
Total Calls Handled: 802.00

Constraint Satisfaction:
Service Level Constraints Satisfied: True
Budget Constraint Satisfied: True
Max Agents per Shift Constraints Satisfied: True

Solver Status and Termination Condition:
Solver Status: ok
Termination Condition: optimal


In [ ]:
from pyomo.environ import *


num_shifts = 3
time_slots_per_shift = 8
time_slots = range(num_shifts * time_slots_per_shift)  # 8 time slots
call_volume_forecast =  [50, 40, 45, 30, 25, 20, 15, 10,
                        20, 30, 40, 50, 60, 70, 80, 90,
                        0, 10, 1, 1, 40, 50, 60, 70] # Example forecast

cost_per_agent = 100  # $ per agent per shift
max_agents_per_shift = 100  # Maximum number of agents allowed per shift
total_budget = 1500
agent_ability = 10  # Number of calls an agent can handle per time slot
call_revenue = 30  # Revenue generated per call answered

# Using Pyomo's Concrete Model:
'''
Used because we are instantiating the model with all the variables already defined (statically).
Used to encase the constraints, variables and objectives
'''
model = ConcreteModel()

# Defining  variables
model.x = Var(range(num_shifts), within=NonNegativeIntegers)  # Number of agents per shift
model.calls_handled = Var(time_slots, within=NonNegativeReals)  # Number of calls handled per time slot

# Objective function: Maximize profit (revenue - cost)
def profit(model):
    total_revenue = sum(model.calls_handled[i] * call_revenue for i in time_slots)
    total_cost = sum(model.x[j] * cost_per_agent for j in range(num_shifts))
    return total_revenue - total_cost

model.obj = Objective(rule=profit, sense=maximize)

# Service level constraint
def service_level_constraint(model, i):
    shift = i // time_slots_per_shift
    return model.calls_handled[i] <= model.x[shift] * agent_ability

# Ensure all calls are handled if possible
def calls_handled_constraint(model, i):
    return model.calls_handled[i] <= call_volume_forecast[i]

# Max agents per shift constraint
def max_agents_constraint(model, j):
    return model.x[j] <= max_agents_per_shift

# Budget constraint
def budget_constraint(model):
    return sum(model.x[j] * cost_per_agent for j in range(num_shifts)) <= total_budget

# Apply constraints
model.service_level = Constraint(time_slots, rule=service_level_constraint)
model.calls_handled_constraint = Constraint(time_slots, rule=calls_handled_constraint)
model.max_agents = Constraint(range(num_shifts), rule=max_agents_constraint)
model.budget = Constraint(rule=budget_constraint)

# Solve the model using GLPK
solver = SolverFactory('glpk')
result = solver.solve(model, tee=False)

# Check solver status and termination condition
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    print("\nOptimal solution found.")
    # Print results
    print("\nOptimal Number of Agents per Shift:")
    for j in range(num_shifts):
        print(f"Shift {j+1}: {model.x[j].value} agents")

    total_cost = sum(cost_per_agent * model.x[j].value for j in range(num_shifts))
    total_revenue = sum(model.calls_handled[i].value * call_revenue for i in time_slots)
    profit = total_revenue - total_cost
    print(f"\nTotal Staffing Cost: ${total_cost:.2f}")
    print(f"Total Revenue: ${total_revenue:.2f}")
    print(f"Profit: ${profit:.2f}")

    # Check constraints
    service_level_satisfied = all(model.calls_handled[i].value <= model.x[i // time_slots_per_shift].value * agent_ability for i in time_slots)
    budget_satisfied = total_cost <= total_budget
    max_agents_satisfied = all(model.x[j].value <= max_agents_per_shift for j in range(num_shifts))

    print("\nConstraint Satisfaction:")
    print(f"Service Level Constraints Satisfied: {service_level_satisfied}")
    print(f"Budget Constraint Satisfied: {budget_satisfied}")
    print(f"Max Agents per Shift Constraints Satisfied: {max_agents_satisfied}")

    print("\nSolver Status and Termination Condition:")
    print(f"Solver Status: {result.solver.status}")
    print(f"Termination Condition: {result.solver.termination_condition}")
elif result.solver.termination_condition == TerminationCondition.infeasible:
    print("\nNo feasible solution found.")
else:
    print("\nSolver encountered an issue. Status: ", result.solver.status)



Optimal solution found.

Optimal Number of Agents per Shift:
Shift 1: 4.0 agents
Shift 2: 7.0 agents
Shift 3: 4.0 agents

Total Staffing Cost: $1500.00
Total Revenue: $24060.00
Profit: $22560.00

Constraint Satisfaction:
Service Level Constraints Satisfied: True
Budget Constraint Satisfied: True
Max Agents per Shift Constraints Satisfied: True

Solver Status and Termination Condition:
Solver Status: ok
Termination Condition: optimal


# Previous Tries

In [ ]:
num_shifts = 4  # Number of shifts in a day
time_slots_per_shift = 6  # Number of time slots per shift
time_slots = range(num_shifts * time_slots_per_shift)  # 24 hours in a day
call_volume_forecast = [50, 40, 45, 30, 25, 20, 15, 10,
                        20, 30, 40, 50, 60, 70, 80, 90,
                        90, 10, 100, 70, 40, 50, 60, 70]  # Example forecast
cost_per_agent = 100  # $20 per agent per shift
max_agents_per_shift = 10  # Maximum number of agents allowed per shift
total_budget = 15000
income_per_call = 15  # Income generated per call answered, Also used as a measure of the agent's capability to handle calls


In [ ]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(range(num_shifts), within=NonNegativeIntegers)  # Number of agents per shift

# Objective function: Maximize profit (income - cost)
def profit(model):
    total_income = sum(call_volume_forecast[i] * income_per_call for i in time_slots)
    total_cost = sum(model.x[j] * cost_per_agent for j in range(num_shifts))
    return total_income - total_cost

model.obj = Objective(rule=profit, sense=maximize)

# Constraints
def service_level_constraint(model, i):
  shift = i // time_slots_per_shift
  # Each agent handles one time slot, so the number of agents should be at least the call volume for that time slot
  # return model.x[shift] >= call_volume_forecast[i]
  return model.x[shift] >= call_volume_forecast[i] / income_per_call


def max_agents_constraint(model, j):
    return model.x[j] <= max_agents_per_shift

def budget_constraint(model):
    # return sum(list(model.x) * cost_per_agent) <= total_budget
    return sum(model.x[j] * cost_per_agent for j in range(num_shifts)) <= total_budget

model.service_level = Constraint(time_slots, rule=service_level_constraint)
model.max_agents = Constraint(range(num_shifts), rule=max_agents_constraint)
model.budget = Constraint(rule=budget_constraint)

# Solve the model using GLPK
solver = SolverFactory('glpk')
result = solver.solve(model, tee=False)

# Check solver status and termination condition
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    print("\nOptimal solution found.")
    # Print results
    print("\nOptimal Number of Agents per Shift:")
    for j in range(num_shifts):
        print(f"Shift {j+1}: {model.x[j].value} agents")

    total_cost = sum(cost_per_agent * model.x[j].value for j in range(num_shifts))
    total_income = sum(call_volume_forecast[i] * income_per_call for i in time_slots)
    profit = total_income - total_cost
    print(f"\nTotal Staffing Cost: ${total_cost:.2f}")
    print(f"Total Income: ${total_income:.2f}")
    print(f"Profit: ${profit:.2f}")

    # Check constraints
    service_level_satisfied = all(model.x[i // time_slots_per_shift].value * time_slots_per_shift >= call_volume_forecast[i] for i in time_slots)
    budget_satisfied = total_cost <= total_budget
    max_agents_satisfied = all(model.x[j].value <= max_agents_per_shift for j in range(num_shifts))

    print("\nConstraint Satisfaction:")
    print(f"Service Level Constraints Satisfied: {service_level_satisfied}")
    print(f"Budget Constraint Satisfied: {budget_satisfied}")
    print(f"Max Agents per Shift Constraints Satisfied: {max_agents_satisfied}")

    print("\nSolver Status and Termination Condition:")
    print(f"Solver Status: {result.solver.status}")
    print(f"Termination Condition: {result.solver.termination_condition}")
elif result.solver.termination_condition == TerminationCondition.infeasible:
    print("\nNo feasible solution found.")
else:
    print("\nSolver encountered an issue. Status: ", result.solver.status)



Optimal solution found.

Optimal Number of Agents per Shift:
Shift 1: 4.0 agents
Shift 2: 4.0 agents
Shift 3: 6.0 agents
Shift 4: 7.0 agents

Total Staffing Cost: $2100.00
Total Income: $17475.00
Profit: $15375.00

Constraint Satisfaction:
Service Level Constraints Satisfied: False
Budget Constraint Satisfied: True
Max Agents per Shift Constraints Satisfied: True

Solver Status and Termination Condition:
Solver Status: ok
Termination Condition: optimal


Optimal Number of Agents per Shift:
- Shift 1: 5.0 agents
- Shift 2: 9.0 agents
- Shift 3: 7.0 agents

- Total Staffing Cost: $1050.00

- Total Income: $9070.00

- Profit: $8020.00


Optimal Number of Agents per Shift:
- Shift 1: 2.0 agents
- Shift 2: 3.0 agents
- Shift 3: 3.0 agents

- Total Staffing Cost: $800.00

- Total Income: $27210.00

- Profit: $26410.00
